In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os 
os.chdir('/Users/bjarnihaukurbjarnason/Documents/genki-signals')

from genki_signals.buffers import DataBuffer

In [ ]:
import time
from threading import Thread

def run_source(source, buffer):
    source.start()
    while source.is_active:
        buffer.extend(source.read())
        time.sleep(1 / 25)

In [ ]:
from genki_signals.data_sources import CameraDataSource, Sampler

buffer = DataBuffer(maxlen=1)

resolution = (480, 360)

camera = CameraDataSource(resolution=resolution)
source = Sampler({"camera": camera}, sample_rate=29)

t = Thread(target=run_source, args=(source, buffer))
t.start()

In [ ]:
buffer.keys()

In [ ]:
buffer["image"].shape

In [ ]:
buffer.plot("image", plot_type="video")

In [ ]:
source.stop(); t.join()

In [ ]:
import genki_signals.signals as s
from genki_signals.system import System

derived = [
    s.Sum('sine_1', 'sine_2', name='sines_sum'),
    s.Sum('sines_sum', 'noise', name='wave_with_noise'),
    s.FourierTransform('wave_with_noise', name='spectrum', window_size=128, window_overlap=64)
]

buffer = DataBuffer(maxlen=400)
system = System(source, derived)

In [ ]:
t = Thread(target=run_source, args=(system, buffer))
t.start()

In [ ]:
buffer

In [ ]:
buffer.plot('wave_with_noise')

In [ ]:
buffer.plot('spectrum', plot_type='spectrogram', window_size=128, sample_rate=100)

In [ ]:
system.stop(); t.join()

In [ ]:
from genki_signals.data_sources import WaveDataSource

wave = WaveDataSource(ble_address='7CEF8BE3-F544-99C9-B25A-3C6DD1281491')

buffer = DataBuffer(maxlen=400)
system = System(wave, [])

In [ ]:
t = Thread(target=run_source, args=(system, buffer))
t.start()

In [ ]:
buffer.plot('gyro')